# Order Processing with OrderQ AI

This notebook demonstrates how to use the OrderQ AI model to process restaurant orders from natural language text into structured JSON.
We will also show how to read input orders from a text file and output the processed results as a TSV file.


In [1]:
import json
import torch
import re
import logging
import pandas as pd
from transformers import T5TokenizerFast, T5ForConditionalGeneration
from typing import Dict, Optional

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

def initialize_model(model_path: str = './trained_model'):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)
    tokenizer = T5TokenizerFast.from_pretrained(model_path)
    return model, tokenizer, device

def validate_and_fix_json(text: str) -> Optional[Dict]:
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        pass
    fixed_text = text.strip()
    if not fixed_text.startswith('{'):
        fixed_text = '{' + fixed_text
    if not fixed_text.endswith('}'):
        fixed_text = fixed_text + '}'
    try:
        fixed_text = re.sub(r'(\w+):', r'"\1":', fixed_text)
        fixed_text = fixed_text.replace("'", '"')
        result = json.loads(fixed_text)
        return result
    except json.JSONDecodeError:
        return None

def process_order(model, tokenizer, device, order_text: str) -> Dict[str, Optional[str]]:
    input_text = 'extract order: ' + order_text
    inputs = tokenizer(input_text, return_tensors='pt', max_length=512, truncation=True, padding=True).to(device)
    outputs = model.generate(**inputs, max_length=256, num_beams=8, early_stopping=True)
    raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    logger.info(f'Raw output: {raw_output}')
    parsed_json = validate_and_fix_json(raw_output)
    if parsed_json:
        return {'status': 'success', 'result': json.dumps(parsed_json)}
    else:
        return {'status': 'error', 'raw_output': raw_output}

def process_orders_from_file(input_file: str, output_file: str, model, tokenizer, device):
    logger.info(f'Reading orders from {input_file}')
    with open(input_file, 'r') as f:
        order_texts = [line.strip() for line in f if line.strip()]
    results = []
    for order_text in order_texts:
        result = process_order(model, tokenizer, device, order_text)
        results.append(result)
    df = pd.DataFrame(results)
    logger.info(f'Saving results to {output_file}')
    df.to_csv(output_file, sep='	', index=False)


## Initialize the model

In [2]:
model_path = './trained_model'
model, tokenizer, device = initialize_model(model_path)


## Process Orders from File

We'll read orders from `orders.txt` and save the structured output to `processed_orders.tsv`

In [4]:
input_file = 'data/orders.txt'
# output_file = 'processed_orders.tsv'
output_file = 'data/processed_orders_02.tsv'

process_orders_from_file(input_file, output_file, model, tokenizer, device)


2025-07-18 03:14:15,354 - INFO - Reading orders from data/orders.txt
2025-07-18 03:14:21,200 - INFO - Raw output: "customer_name": "John Smith", "order_type": "delivery", "total_number_of_different_items": 2, "order_items_name": "pizza|diet cokes", "order_items_quantity": "2|3", "order_items_modifications": "large|extra cheese|", "order_notes": null
2025-07-18 03:14:27,328 - INFO - Raw output: "customer_name": "Sarah Johnson", "order_type": "takeout", "total_number_of_different_items": 2, "order_items_name": "burger|coffees": "1|2", "order_items_quantity": "1|2", "order_items_modifications": "|no onions|", "order_notes": null
2025-07-18 03:14:33,198 - INFO - Raw output: "customer_name": "Mike Davis", "order_type": "delivery", "total_number_of_different_items": 2, "order_items_name": "chicken sandwiches|extra pickles", "order_items_quantity": "3|4", "order_items_modifications": "|extra pickles", "order_notes": null
2025-07-18 03:14:39,807 - INFO - Raw output: "customer_name": "Anna Wils